In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, \
    HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from htools import *
from jabberwocky.openai_utils import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [3]:
cd_root()

Current directory: /Users/hmamin/roboduck


In [4]:
os.environ['OPENAI_API_KEY'] = api_key = load_openai_api_key()

In [5]:
print(load_prompt('debug')['prompt'])

debug: Could try davinci text as well but codex is free for now. You may want to strip triple double-quotes from the end in case codex generates them (we don't include that as a stop phrase because codex might generate a docstring as part of a correct code snippet).
-------------------------------------------------------------------------------

"""ANSWER KEY

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it. In the section titled SOLUTION PART 2, write a corrected version of the input code snippet. If you don't know what the problem is, SOLUTION PART 1 should list a few possible causes or things I could try in order to identify the issue and SOLUTION PART 2 should say N/A. Be concise and use simple language because I am a begin

In [6]:
system_prompt_text = """You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.
"""
system_prompt = SystemMessage(content=system_prompt_text)

In [7]:
user_prompt_text = """This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
{question}

CURRENT CODE SNIPPET:
{code}

LOCAL VARIABLES:
{local_vars}

GLOBAL VARIABLES:
{global_vars}"""
user_prompt_template = HumanMessagePromptTemplate.from_template(
    user_prompt_text
)

In [8]:
kwargs = {
    'question': 'Why will this throw an index error soon?',
    'code': """def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums)):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums""",
    'local_vars': """{
    'nums': [3, 4, 2, 1, 5, 9],   # type: list
    'i': 0,   # type: int
    'j': 4,   # type: int
}""",
    'global_vars': """{
}"""
}

In [9]:
messages = [
    system_prompt,
    user_prompt_template.format(**kwargs)
]

In [10]:
print('\n'.join(m.content for m in messages))

You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English ex

In [358]:
chat = ChatOpenAI(temperature=.66, a='b', max_tokens=33, 
                  model_name='gpt-3.5-turbo')
chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fed643756d0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', model_kwargs={'temperature': 0.66, 'a': 'b'}, openai_api_key=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=33)

In [359]:
chat.model_kwargs

{'temperature': 0.66, 'a': 'b'}

In [360]:
chat.max_tokens

33

In [58]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The code will throw an index error soon because the inner loop is iterating up to the length of the list, which means that on the last iteration, `nums[j + 1]` will be out of range. To fix this, we need to change the range of the inner loop to `range(len(nums) - i - 1)`.

SOLUTION PART 2:

```
def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums) - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


In [60]:
messages.append(res)
messages.append(
    user_prompt_template.format(
        **{**kwargs, 
           'question': 'Can you revise your solution so you only find the length of nums once?'}
    )
)

In [62]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The problem with the current code is that it is calling `len(nums)` twice in the inner loop, which is inefficient. To fix this, we can store the length of `nums` in a variable before the loop and use that variable instead.

SOLUTION PART 2:

```
def bubble_sort(nums):
    n = len(nums)
    for i in range(n):
        for j in range(n - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


Took first stab at storing this info in a file (py for now). Try loading.

In [5]:
import importlib
from langchain.schema import AIMessage
from roboduck.prompts import load_template
from inspect import Parameter

In [6]:
class DummyChatModel:
    
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        
    def __call__(self, messages, stop=None):
        return AIMessage(content=messages[-1].content.upper())

In [7]:
# Old approach. Decorator is not as useful for our use case because we want to
# define the base function once but create several variants of it using
# different fields.
# def add_kwargs(*fields):
#     def decorator(func):
#         # In practice langchain checks for this anyway if we ask for a
#         # completion, but outside of that context we need typecheck
#         # because otherwise we could provide no kwargs and _func wouldn't
#         # complain. Just use generic type because we only care that a value is
#         # provided.
#         @typecheck(**{f: object for f in fields})
#         @wraps(func)
#         def wrapper(**kwargs):
#             return func(**kwargs)

#         sig = signature(wrapper)
#         params_ = {field: Parameter(field, Parameter.KEYWORD_ONLY)
#                    for field in fields}
#         wrapper.__signature__ = sig.replace(parameters=params_.values())
#         return wrapper
#     return decorator


# @add_kwargs('question', 'abc')
# def _reply(**kwargs):
#     return kwargs

In [8]:
def add_kwargs(func, fields, hide_fields=(), strict=False):
    # Hide_fields must have default values in existing function. They will not
    # show up in the new docstring and the user will not be able to pass in a
    # value when calling the new function - it will always use the default.
    # To set different defaults, you can pass in a partial rather than a 
    # function as the first arg here.
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)

    if hide_fields and not strict:
        raise ValueError(
            'You must set strict=True when providing one or more '
            'hide_fields. Otherwise the user can still pass in those args.'
        )
    sig = signature(wrapper)
    params_ = {k: v for k, v in sig.parameters.items()}
    
    # Remove any fields we want to hide.
    for field in hide_fields:
        if field not in params_:
            warnings.warn(f'No need to hide field {field} because it\'s not '
                          'in the existing function signature.')
        elif params_.pop(field).default == Parameter.empty:
            raise TypeError(
                f'Field "{field}" is not a valid hide_field because it has '
                'no default value in the original function.'
            )
            
    if getattr(params_.pop('kwargs', None), 'kind') != Parameter.VAR_KEYWORD:
        raise TypeError(f'Function {func} must accept **kwargs.')
    new_params = {
        field: Parameter(field, Parameter.KEYWORD_ONLY)
        for field in fields
    }
    overlap = set(new_params) & set(params_)
    if overlap:
        raise RuntimeError(
            f'Some of the kwargs you tried to inject into {func} already '
            'exist in its signature. This is not allowed because it\'s '
            'unclear how to resolve default values and parameter type.'
        )

    params_.update(new_params)
    wrapper.__signature__ = sig.replace(parameters=params_.values())
    if strict:
        # In practice langchain checks for this anyway if we ask for a
        # completion, but outside of that context we need typecheck
        # because otherwise we could provide no kwargs and _func wouldn't
        # complain. Just use generic type because we only care that a value is
        # provided.
        wrapper = typecheck(wrapper, **{f: object for f in fields})
    return wrapper

def _reply(key, bar=-1, **kwargs):
    """Test docstring."""
    return key, bar, kwargs

reply = add_kwargs(_reply, ['statement', 'response'])
strict_reply = add_kwargs(_reply, ['statement', 'response'], strict=True)
with assert_raises(TypeError):
    no_key_reply = add_kwargs(_reply, ['statement', 'response'],
                              hide_fields=['key'], strict=True)
    
    
def reply_(key='aaa', bar=-1, **kwargs):
    """Test docstring."""
    return key, bar, kwargs


no_key_reply = add_kwargs(reply_, ['statement', 'response'],
                          hide_fields=['key'], strict=True)
diff_key_reply = add_kwargs(
    partial(reply_, key='zzz'), ['statement', 'response'],
    hide_fields=['key'], strict=True
)

As expected, got TypeError(Field "key" is not a valid hide_field because it has no default value in the original function.).


In [9]:
reply('a', bar=99, statement=44, response=-1)

('a', 99, {'statement': 44, 'response': -1})

In [10]:
reply('a', bar=99, statement=44)

('a', 99, {'statement': 44})

In [11]:
strict_reply('a', bar=99, statement=44, response=-1)

('a', 99, {'statement': 44, 'response': -1})

In [12]:
with assert_raises(TypeError):
    strict_reply('a', bar=99, statement=44)

As expected, got TypeError(missing a required argument: 'response').


In [13]:
no_key_reply(bar=99, statement=44, response=-1)

('aaa', 99, {'statement': 44, 'response': -1})

In [14]:
with assert_raises(TypeError):
    no_key_reply(bar=99, statement=44, response=-1, key=99)

As expected, got TypeError(got an unexpected keyword argument 'key').


In [15]:
diff_key_reply(bar=99, statement=44, response=-1)

('zzz', 99, {'statement': 44, 'response': -1})

In [16]:
with assert_raises(TypeError):
    diff_key_reply(bar=99, statement=44, response=-1, key='eeee')

As expected, got TypeError(got an unexpected keyword argument 'key').


In [40]:
class Chat:
    
    def __init__(self, system, user, model_kwargs=None, 
                 chat_class=ChatOpenAI, history=(), **chat_kwargs):
        # model_kwargs affect completion directly 
        # (e.g. 'temperature' or 'top_p').
        # chat_kwargs contains things like `callback_manager` or `verbose`.
        self.model_kwargs = dict(model_kwargs or {})
        self.chat_kwargs = dict(chat_kwargs)
        self.chat = chat_class(**self.chat_kwargs, **self.model_kwargs)
        self.system_message = SystemMessage(content=system)
        if isinstance(user, str):
            user = {'reply': user}
        self.user_templates = {
            k: HumanMessagePromptTemplate.from_template(v)
            for k, v in user.items()
        }
        self.default_user_key = next(iter(self.user_templates))
        self.default_user_fields = (self.user_templates[self.default_user_key]
                                    .input_variables)
        self._history = list(history) or [self.system_message]
        self._create_reply_methods()
        
    def _create_reply_methods(self):
        """Creates two options for user to send replies:
        1. call chat.reply(), using the key_ arg to determine which type of
        user_message is sent. The docstring shows the default user 
        message type's fields but if you set the key accordingly you can pass
        in fields for another message type. We choose not to infer key_
        because some user_message types may accept the same fields.
        2. call methods like chat.question() or chat.statement(), where 'chat'
        and 'statement' are the names of all available user message types
        (i.e. the keys of the `user` dict in the prompt config file). You can
        not pass in key_.
        """
        for k, v in self.user_templates.items():
            if hasattr(self, k):
                warnings.warn(
                    f'Name collision: prompt defines user message type {k} '
                    f'but Chat class already has a method with that name. '
                    f'Method will be named {k}_ instead.'
                )
                k = k + '_'
            meth = add_kwargs(partial(self._reply, key_=k), 
                              fields=v.input_variables,
                              hide_fields=['key_'],
                              strict=True)
            setattr(self, k, meth)
        setattr(
            self, 
            'reply', 
            add_kwargs(self._reply, self.default_user_fields, strict=False)
        )
        
    @classmethod
    def from_config(cls, name, model_kwargs=None, **chat_kwargs):
        template = load_template(name)
        if model_kwargs:
            template['model_kwargs'].update(model_kwargs)
        return cls(**template, **chat_kwargs)
        
    def _user_message(self, *, key_='', **kwargs):
        key = key_ or self.default_user_key
        template = self.user_templates[key]
        return template.format(**kwargs)
    
    def _reply(self, *, key_='', **kwargs):
        print('_reply:', key_, kwargs)
        user_message = self._user_message(key_=key_, **kwargs)
        self._history.append(user_message)
        try:
            response = self.chat(self._history)
        except Exception as e:
            self._history.pop(-1)
            raise e
        self._history.append(response)
        return response
    
    def history(self, sep='\n\n', speaker_prefix=True):
        """Return chat history as a single string."""
        res = []
        for row in self._history:
            reply = row.content
            if speaker_prefix:
                speaker = type(row).__name__.split('Message')[0]
                reply = f'{speaker}: {reply}'
            res.append(reply)
        return sep.join(res)

In [20]:
chat = Chat.from_config('debug', chat_class=DummyChatModel)

In [21]:
tmp = chat._user_message(
    code='a = 3\nb = 4', question='Why?', local_vars='{3: 4}',
    global_vars='{True: False}', next_line='b = 4'
)

print(tmp.content)

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
Why?

CURRENT CODE SNIPPET:
a = 3
b = 4

NEXT LINE:
b = 4

LOCAL VARIABLES:
{3: 4}

GLOBAL VARIABLES:
{True: False}


In [22]:
tmp = chat._user_message(
    key_='contextless',
    question='Why?'
)

print(tmp.content)

QUESTION:
Why?


In [23]:
chat.contextless(question='x')

_reply: contextless {'question': 'x'}


AIMessage(content='QUESTION:\nX', additional_kwargs={})

In [24]:
with assert_raises(TypeError):
    chat.contextless(question='x', key_='contextful')

As expected, got TypeError(got an unexpected keyword argument 'key_').


In [25]:
chat.contextful(
    code='a = 3\nb = 4', question='Why?', local_vars='{3: 4}',
    global_vars='{True: False}', next_line='b = 4'
)

_reply: contextful {'code': 'a = 3\nb = 4', 'question': 'Why?', 'local_vars': '{3: 4}', 'global_vars': '{True: False}', 'next_line': 'b = 4'}


AIMessage(content="THIS CODE SNIPPET IS NOT WORKING AS EXPECTED. HELP ME DEBUG IT. FIRST READ MY QUESTION, THEN EXAMINE THE SNIPPET OF CODE THAT IS CAUSING THE ISSUE AND LOOK AT THE VALUES OF THE LOCAL AND GLOBAL VARIABLES. YOUR RESPONSE MUST HAVE EXACTLY TWO PARTS. IN THE SECTION TITLED SOLUTION PART 1, USE PLAIN ENGLISH TO EXPLAIN WHAT THE PROBLEM IS AND HOW TO FIX IT (IF YOU DON'T KNOW WHAT THE PROBLEM IS, SOLUTION PART 1 SHOULD INSTEAD LIST A FEW POSSIBLE CAUSES OR THINGS I COULD TRY IN ORDER TO IDENTIFY THE ISSUE). IN THE SECTION TITLED SOLUTION PART 2, WRITE A CORRECTED VERSION OF THE INPUT CODE SNIPPET (IF YOU DON'T KNOW, SOLUTION PART 2 SHOULD SAY NONE). SOLUTION PART 2 MUST CONTAIN ONLY PYTHON CODE - THERE MUST NOT BE ANY ENGLISH EXPLANATION OUTSIDE OF CODE COMMENTS OR DOCSTRINGS. BE CONCISE AND USE SIMPLE LANGUAGE BECAUSE I AM A BEGINNING PROGRAMMER.\n\nQUESTION:\nWHY?\n\nCURRENT CODE SNIPPET:\nA = 3\nB = 4\n\nNEXT LINE:\nB = 4\n\nLOCAL VARIABLES:\n{3: 4}\n\nGLOBAL VARIABLES:

In [26]:
chat.reply(
    code='a = 3\nb = 4', question='Why?', local_vars='{3: 4}',
    global_vars='{True: False}', next_line='b = 4'
)

_reply:  {'code': 'a = 3\nb = 4', 'question': 'Why?', 'local_vars': '{3: 4}', 'global_vars': '{True: False}', 'next_line': 'b = 4'}


AIMessage(content="THIS CODE SNIPPET IS NOT WORKING AS EXPECTED. HELP ME DEBUG IT. FIRST READ MY QUESTION, THEN EXAMINE THE SNIPPET OF CODE THAT IS CAUSING THE ISSUE AND LOOK AT THE VALUES OF THE LOCAL AND GLOBAL VARIABLES. YOUR RESPONSE MUST HAVE EXACTLY TWO PARTS. IN THE SECTION TITLED SOLUTION PART 1, USE PLAIN ENGLISH TO EXPLAIN WHAT THE PROBLEM IS AND HOW TO FIX IT (IF YOU DON'T KNOW WHAT THE PROBLEM IS, SOLUTION PART 1 SHOULD INSTEAD LIST A FEW POSSIBLE CAUSES OR THINGS I COULD TRY IN ORDER TO IDENTIFY THE ISSUE). IN THE SECTION TITLED SOLUTION PART 2, WRITE A CORRECTED VERSION OF THE INPUT CODE SNIPPET (IF YOU DON'T KNOW, SOLUTION PART 2 SHOULD SAY NONE). SOLUTION PART 2 MUST CONTAIN ONLY PYTHON CODE - THERE MUST NOT BE ANY ENGLISH EXPLANATION OUTSIDE OF CODE COMMENTS OR DOCSTRINGS. BE CONCISE AND USE SIMPLE LANGUAGE BECAUSE I AM A BEGINNING PROGRAMMER.\n\nQUESTION:\nWHY?\n\nCURRENT CODE SNIPPET:\nA = 3\nB = 4\n\nNEXT LINE:\nB = 4\n\nLOCAL VARIABLES:\n{3: 4}\n\nGLOBAL VARIABLES:

In [27]:
chat.reply(question='How are you?', key_='contextless')

_reply: contextless {'question': 'How are you?'}


AIMessage(content='QUESTION:\nHOW ARE YOU?', additional_kwargs={})

In [28]:
print(chat.history())

System: You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

Human: QUESTION:
x

AI: QUESTION:
X

Human: This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain

In [85]:
# This would make more sense if our messages included speakers, i.e. if a
# user_message looked like 'Me: {reply}' and gpt was prompted to reply like
# 'Robert Sapolsky: {reply}' (for example).
print(chat.history(speaker_prefix=False))

You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

QUESTION:
How are you?

QUESTION:
HOW ARE YOU?


Error when using real ChatOpenAI obj.

In [34]:
chat = Chat.from_config('debug')
chat.chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fdb1f1eb5e0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', model_kwargs={'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0.2, 'presence_penalty': 0.0, 'logit_bias': {37811: -100, 27901: -50}, 'stop': ['QUESTION', 'SOLUTION PART 1', 'SOLUTION PART 3']}, openai_api_key=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=512)

In [35]:
chat.model_kwargs

{'model_name': 'gpt-3.5-turbo',
 'temperature': 0.0,
 'top_p': 0.99,
 'max_tokens': 512,
 'frequency_penalty': 0.2,
 'presence_penalty': 0.0,
 'logit_bias': {37811: -100, 27901: -50},
 'stop': ['QUESTION', 'SOLUTION PART 1', 'SOLUTION PART 3']}

In [ ]:
chat.

In [25]:
chat.reply(question='How are you?', key_='contextless')

AIMessage(content="As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any programming-related questions you may have. How can I help you today?", additional_kwargs={})

In [26]:
print(chat.history())

System: You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

Human: QUESTION:
How are you?

AI: As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any programming-related questions you may have. How can I help you today?


In [38]:
chat = Chat.from_config('debug', {'temperature': .7})
chat.chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fdb1f1eb5e0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', model_kwargs={'temperature': 0.7, 'top_p': 0.99, 'frequency_penalty': 0.2, 'presence_penalty': 0.0, 'logit_bias': {37811: -100, 27901: -50}, 'stop': ['QUESTION', 'SOLUTION PART 1', 'SOLUTION PART 3']}, openai_api_key=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=512)

In [39]:
chat.model_kwargs

{'model_name': 'gpt-3.5-turbo',
 'temperature': 0.7,
 'top_p': 0.99,
 'max_tokens': 512,
 'frequency_penalty': 0.2,
 'presence_penalty': 0.0,
 'logit_bias': {37811: -100, 27901: -50},
 'stop': ['QUESTION', 'SOLUTION PART 1', 'SOLUTION PART 3']}

## Signature surgery

Inintial prototype for inserting fields into signature and docstring.

In [107]:
class Foo:
    def __init__(self, fields, name2fields):
        self.reply = self._make_func(self._reply, fields)
        for k, v in name2fields.items():
            setattr(self, k, self._make_func(self._reply, v))

    def _reply(self, **kwargs):
        print('Calling _reply')
        return {'kwargs': kwargs, 'completion': 'new text...'}
    
    def _make_func(self, func, fields):
        # In practice I think langchain checks for this anyway if we ask for a
        # completion, but outside of that context typecheck would be necessary
        # because otherwise we can provide no kwargs and _func won't complain. 
        @typecheck(**{f: str for f in fields})
        @wraps(func)
        def wrapper(**kwargs):
            return func(**kwargs)
        
        sig = signature(wrapper)
        params_ = {field: Parameter(field, Parameter.KEYWORD_ONLY)
                   for field in fields}
        wrapper.__signature__ = sig.replace(parameters=params_.values())
        return wrapper

In [100]:
f = Foo(
    ['a', 'dog', 'x'],
    {'question': ['fact', 'question', 'answer'],
     'statement': ['salutation', 'name']}
)

In [101]:
f.reply

<function __main__.Foo._reply(*, a, dog, x)>

In [102]:
f.question

<function __main__.Foo._reply(*, fact, question, answer)>

In [103]:
f.statement

<function __main__.Foo._reply(*, salutation, name)>

In [104]:
with assert_raises(TypeError):
    f.statement()

As expected, got TypeError(missing a required argument: 'salutation').


In [105]:
f.statement(salutation='hi', name='harry')

Calling _reply


{'kwargs': {'salutation': 'hi', 'name': 'harry'}, 'completion': 'new text...'}

In [106]:
f.question(fact='birds are sad', question='why?', answer='yes')

Calling _reply


{'kwargs': {'fact': 'birds are sad', 'question': 'why?', 'answer': 'yes'},
 'completion': 'new text...'}

## Debug scratch

See if we can use frames to identify whether we need to provide context for a user message (i.e. if frame has changed since we last did).

In [4]:
from roboduck.debugger import duck

In [70]:
def binary_search(x, nums):
    if not nums:
        return -1
    duck(backend='repeat')
    mid = len(nums) // 2
    if x == nums[mid]:
        return x
    if x > nums[mid]:
        return binary_search(x, nums[mid + 1:])
    if x < nums[mid]:
        return binary_search(x, nums[:mid])

In [71]:
nums = [33, 44, 55, 66, 77, 88, 99, 111]

In [72]:
binary_search(3, nums)

> <ipython-input-70-8a37149461d4>(5)binary_search()
-> mid = len(nums) // 2
>>> l .
  1  	def binary_search(x, nums):
  2  	    if not nums:
  3  	        return -1
  4  	    duck(backend='repeat')
  5  ->	    mid = len(nums) // 2
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid]:
 11  	        return binary_search(x, nums[:mid])
>>> y?
next line:     mid = len(nums) // 2
[Duck] """>>> n
> <ipython-input-70-8a37149461d4>(6)binary_search()
-> if x == nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(8)binary_search()
-> if x > nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(10)binary_search()
-> if x < nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(11)binary_search()
-> return binary_search(x, nums[:mid])
>>> l .
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid

BdbQuit: 

In [13]:
binary_search(33, nums)

33

In [14]:
binary_search(39, nums)

-1

In [15]:
binary_search(111, nums)

111

In [16]:
binary_search(112, nums)

-1

In [73]:
def test():
    for i in range(5):
        print(i)
        duck(backend='repeat')

In [74]:
test()

0
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
0
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> i
1
>>> y?
frmae_id 140192236901280
next line:         print(i)
[Duck] """>>> n
1
> <ipython-input-73-943befe6b744>(4)test()
-> duck(backend='repeat')
>>> i
1
>>> n
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
1
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> i
1
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> l .
  1  	def test():
  2  	    for i in range(5):
  3  ->	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280


BdbQuit: 